Verifies optimex results with static inputs vs conventional LCA

In [1]:
import bw2data as bd
import bw2calc as bc
import logging

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s  - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

bd.projects.set_current("optimex_ma")
bd.databases

Databases dictionary with 9 object(s):
	biosphere3
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei3.10-SSP2-RCP19-2020
	ei3.10-SSP2-RCP19-2030
	ei3.10-SSP2-RCP19-2040
	ei3.10-SSP2-RCP19-2050
	foreground
	foreground_ver

In [2]:
ecoinvent_vs = "3.10"

eidb = bd.Database(f"ecoinvent-{ecoinvent_vs}-cutoff")

In [3]:
if "foreground_ver" in bd.databases:
    del bd.databases["foreground_ver"] # to make sure we create the foreground from scratch
foreground = bd.Database("foreground_ver")
foreground.write({})
foreground.register()

In [4]:
original_bg_activities_el = {
    "wind_onshore": eidb.get(name="electricity production, wind, 1-3MW turbine, onshore", location="DE"),
    "photovoltaic": eidb.get(name="electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted", location="DE"), 
}

for key, act in original_bg_activities_el.items():
    fg_act = act.copy(code=f"elec_prod_{key.replace(' ','_')}", database=foreground.name)
    fg_act["name"] = f"electricity production, {key}, time-explicit"
    fg_act["functional flow"] = "electricity"
    fg_act.save()

[act for act in foreground]

['electricity production, photovoltaic, time-explicit' (kilowatt hour, DE, None),
 'electricity production, wind_onshore, time-explicit' (kilowatt hour, DE, None)]

In [5]:
method = ('ecoinvent-3.10', 'EF v3.1', 'climate change', 'global warming potential (GWP100)')

In [6]:
original_act_gwp = {}
for name, act in original_bg_activities_el.items():
    lca = bc.LCA({act: 1}, method)
    lca.lci()
    lca.lcia()
    original_act_gwp[name] = lca.score
original_act_gwp

2025-01-25 15:54:48  - INFO - Initialized LCA object. Demand: {140457060773036032: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x000002263AB97610>, <bw_processing.datapackage.Datapackage object at 0x000002263AB6F390>, <bw_processing.datapackage.Datapackage object at 0x000002263AB16F10>]
2025-01-25 15:54:55  - INFO - Initialized LCA object. Demand: {140456929709424640: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x000002263ABCB0D0>, <bw_processing.datapackage.Datapackage object at 0x000002263ABECF90>, <bw_processing.datapackage.Datapackage object at 0x000002263D368510>]


{'wind_onshore': 0.020787445708147764, 'photovoltaic': 0.10114293937837837}

In [7]:
import numpy as np
import bw_temporalis as bwt


lifetime_dict = {
    "wind_onshore": 9,
    "photovoltaic": 9,
}


In [8]:
#TODO: check for potential improvements for mapping quality based on metadata

def set_temporal_distribution(activity: bd.backends.proxies.Activity, lifetime):
    td_use_phase = bwt.easy_timedelta_distribution(
        start=0,
        end=lifetime,
        resolution="Y",
        steps=(lifetime + 1),
        kind="uniform",
    )
    for exc in activity.exchanges():
        exc["temporal_distribution"] = td_use_phase
        exc.save()
    return activity

for act in foreground:
    key = next((k for k in lifetime_dict if k in act['name'].lower()), None)
    if key:
        lifetime = lifetime_dict[key]
    set_temporal_distribution(act,  lifetime)
    act.save()

In [9]:
import numpy as np

demand_el = np.ones(30)

years = np.arange(2020, 2050)

td_demand_el = bwt.TemporalDistribution(
    date=np.arange(2020-1970, 30, dtype="datetime64[Y]"), 
    amount=demand_el
)

demand_input = {
    "electricity": td_demand_el,
}

In [10]:
from datetime import datetime

optimex_type = "static_ver"

database_date_dict = {
    eidb.name: datetime.strptime("2020", "%Y"),   
    foreground.name: "dynamic", # flag databases that should be temporally distributed with "dynamic"
}

In [11]:
from optimex import optimex

start_date = datetime.strptime("2020", "%Y")
timehorizon = 30

opt = optimex.Optimex(
    demand = demand_input,
    start_date=start_date,
    method=method,
    database_date_dict = database_date_dict,
    timehorizon=timehorizon,
)

In [12]:
technosphere_tensor, biosphere_tensor, production_tensor = opt.construct_foreground_tensors()
demand_matrix = opt.parse_demand()
inventory_tensor = opt.sequential_inventory_tensor_calculation(cutoff=1e4)
mapping_matrix = opt.construct_mapping_matrix()
characterization_matrix = opt.construct_characterization_matrix(dynamic=False, metric="GWP")

2025-01-25 15:55:04  - INFO - Calculating inventory for database: ecoinvent-3.10-cutoff
2025-01-25 15:55:05  - INFO - Initialized LCA object. Demand: {140456841960390657: 1, 140456784733306885: 1, 140456964169826304: 1, 140457103177449473: 1, 140457238791880706: 1, 140456883240730625: 1, 140456910386266114: 1, 140457071539814401: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x000002263CAC6650>, <bw_processing.datapackage.Datapackage object at 0x000002263AC1C110>, <bw_processing.datapackage.Datapackage object at 0x000002263AC3CD50>]
2025-01-25 15:55:12  - INFO - Factorized LCI for database: ecoinvent-3.10-cutoff
100%|██████████| 8/8 [01:05<00:00,  8.13s/it]
2025-01-25 15:56:17  - INFO - Finished calculating inventory for database: ecoinvent-3.10-cutoff


In [13]:
from optimex.converter import Converter

converter = Converter(opt)

In [14]:
unconstrained_inputs = converter.combine_and_check()
converter.pickle_model_inputs(f"verification_opt_inputs.pkl")

In [15]:
from optimex import optimizer

model = optimizer.create_model(unconstrained_inputs, name = f"optimex_ver_elec")


2025-01-25 15:56:19  - INFO - Creating sets
2025-01-25 15:56:19  - INFO - Creating parameters
2025-01-25 15:56:19  - INFO - Creating variables
2025-01-25 15:56:19  - INFO - Creating expressions
2025-01-25 15:56:29  - INFO - Creating constraints
2025-01-25 15:56:29  - INFO - Creating objective function


In [16]:
model, results = optimizer.solve_model(model, compute_iis=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-10
Read LP format model from file C:\Users\HP\AppData\Local\Temp\tmph443tjs_.pyomo.lp
Reading time = 0.02 seconds
x1: 92 rows, 60 columns, 630 nonzeros
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8365U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 92 rows, 60 columns and 630 nonzeros
Model fingerprint: 0x0427ccd3
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [2e-03, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 72 rows and 31 columns
Presolve time: 0.00s
Presolved: 20 rows, 29 columns, 200 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0559612e-01   8.000000e+01   0.000000e+00     

In [17]:
from optimex.postprocessing import PostProcessor

pp = PostProcessor(model)
pp.get_scaling()

Process,elec_prod_photovoltaic,elec_prod_wind_onshore
Time,,
2020,0.0,10.0
2030,0.0,10.0
2040,0.0,10.0


In [27]:
conventional = original_act_gwp["wind_onshore"] * 30
time_explicit = model.OBJ()

In [29]:
error = (conventional - time_explicit) / conventional
error

0.010960172435894755